In [2]:
# ============================================
# 📦 1. Install dependencies
# ============================================
!pip install -q transformers accelerate datasets peft bitsandbytes

# Check GPU
!nvidia-smi


Wed Dec  3 17:20:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "sentiment")
train = dataset["train"]
test = dataset["test"]

train = train.shuffle(seed=42).select(range(3000))
test = test.shuffle(seed=42).select(range(500))

In [4]:
# ============================================
# 3. Prepare text → prompt format
# ============================================

from datasets import load_dataset

# tiny + loads instantly
dataset = load_dataset("tweet_eval", "sentiment")

train_ds = dataset["train"]
test_ds  = dataset["test"]

# shrink size for fast Colab finetune
train_ds = train_ds.shuffle(seed=42).select(range(3000))
test_ds  = test_ds.shuffle(seed=42).select(range(500))

label_map = {0: "negative", 1: "neutral", 2: "positive"}

def format_example(row):
    text = row["text"]
    label = label_map[row["label"]]

    return {
        "prompt": f"Tweet:\n{text}\nSentiment:",
        "target": f" {label}"
    }

train_ds = train_ds.map(format_example)
test_ds  = test_ds.map(format_example)

train_ds[0]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'text': 'Few more hours to iPhone 6s launch and im still using the 4th generation ^_^',
 'label': 2,
 'prompt': 'Tweet:\nFew more hours to iPhone 6s launch and im still using the 4th generation ^_^\nSentiment:',
 'target': ' positive'}

In [8]:
# ---------- Tokenization (make labels) ----------
from transformers import AutoTokenizer

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    texts = [p + t for p, t in zip(batch["prompt"], batch["target"])]
    tok = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    # make labels: same as input_ids (Trainer will compute causal loss)
    tok["labels"] = [list(ids) for ids in tok["input_ids"]]
    return tok

train_tok = train_ds.map(tokenize, batched=True)
test_tok  = test_ds.map(tokenize, batched=True)

# Remove originals if you want
train_tok = train_tok.remove_columns(["text", "prompt", "target", "label"])
test_tok  = test_tok.remove_columns(["text", "prompt", "target", "label"])

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
# ---------- Load model (no device_map) ----------
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,   # keep fp16 for Colab/T4
    # DO NOT use load_in_4bit or device_map here if you want Trainer to work reliably
)

# attach LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"]
)
model = get_peft_model(model, peft_config)

# IMPORTANT: disable use_cache during training to ensure loss is returned
model.config.use_cache = False

model.print_trainable_parameters()

trainable params: 811,008 || all params: 82,723,584 || trainable%: 0.9804


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [10]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,   # causal lm
)

training_args = TrainingArguments(
    output_dir="distilgpt2-lora",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    num_train_epochs=2,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    fp16=True,
    logging_steps=50,
    save_steps=500,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    data_collator=data_collator,   # <- important
)

trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,5.003800
100,4.201900
150,3.971400
200,3.919500
250,3.937500
300,3.883100
350,3.892800
400,3.832300
450,3.806900
500,3.817000


TrainOutput(global_step=750, training_loss=3.9571689453125, metrics={'train_runtime': 137.024, 'train_samples_per_second': 43.788, 'train_steps_per_second': 5.473, 'total_flos': 199709687808000.0, 'train_loss': 3.9571689453125, 'epoch': 2.0})

In [11]:
# ============================================
# 🎯 7. Test model generation
# ============================================
import torch

def gen(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    out = model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.7,
    )
    return tokenizer.decode(out[0], skip_special_tokens=True)

print(gen("Review:\nThe food was amazing and the service was fast.\nSentiment:"))
print(gen("Review:\nThis place was terrible, worst experience.\nSentiment:"))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Review:
The food was amazing and the service was fast.
Sentiment: positive
Sentiment: positive
Sentiment: positive
Sentiment: positive
Sentiment: positive
Sentiment: positive
Sentiment: positive
Sentiment: positive
Sentiment: positive
Sentiment: positive
Sentiment:
Review:
This place was terrible, worst experience.
Sentiment: negative
Sentiment: negative
Sentiment: negative
Sentiment: negative
Sentiment: negative
Sentiment: negative
Sentiment: negative
Sentiment: negative
Sentiment: negative
Sentiment: negative
Sentiment:


In [12]:
# ============================================
# 💾 8. Save LoRA adapter
# ============================================
model.save_pretrained("distilgpt2-finetuned-lora")
print("Saved!")

Saved!
